# PyTorch: Optimization & Training

https://github.com/pytorch/examples/tree/master/mnist

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np

In [2]:
seed = 1 # shuffle을 아래서 할 것

batch_size = 64
test_batch_size = 64

no_cuda = False

In [3]:
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Preprocess

In [4]:
torch.manual_seed(seed)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)


test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True)

# Model

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Optimization

- Model과 Optimization 설정

In [7]:
model = Net().to(device) # 위의 class

In [8]:
# parmaneter : __init__의 내용물
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)

In [10]:
# tensorflow의 summary 대신
params = list(model.parameters())
for i in range(8):
    print(params[i].size())
    
# conv1의 w, b, conv2의 w, b, fc1의 w, b, fc2의 w, b (순서대로)

torch.Size([20, 1, 5, 5])
torch.Size([20])
torch.Size([50, 20, 5, 5])
torch.Size([50])
torch.Size([500, 800])
torch.Size([500])
torch.Size([10, 500])
torch.Size([10])


# Before Training

- 학습하기 전에 Model이 Train할 수 있도록 Train Mode로 변환
    - Convolution 또는 Linear 뿐만 아니라, DropOut과 추후에 배우게 될 Batch Normalization과 같이 parameter를 가진 Layer들도 학습하기 위해 준비

In [11]:
# train mode와 evaluation mode가 있음 
model.train()  # train mode

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

- 모델에 넣기 위한 첫 Batch 데이터 추출

In [12]:
data, target = next(iter(train_loader))

In [13]:
# image, label
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

- 추출한 Batch 데이터를 cpu 또는 gpu와 같은 device에 compile

In [14]:
data, target = data.to(device), target.to(device)

In [15]:
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

- gradients를 clear해서 새로운 최적화 값을 찾기 위해 준비

In [16]:
# optimizer 초기화
optimizer.zero_grad()

- 준비한 데이터를 model에 input으로 넣어 output을 얻음

In [18]:
# output = prediction
output = model(data)

- Model에서 예측한 결과를 Loss Function에 넣음
    - 여기 예제에서는 Negative Log-Likelihood Loss 라는 Loss Function을 사용

In [19]:
# output = y_pred, target = y(label)
loss = F.nll_loss(output, target)

#### 한 세트 (loss 계산 > gradients 계산 > parameter update)

- Back Propagation을 통해 Gradients를 계산

In [20]:
loss.backward()

- 계산된 Gradients는 계산된 걸로 끝이 아니라 Parameter에 Update

In [20]:
optimizer.step()

# Start Training

위의 최적화 과정을 반복하여 학습 시작

In [21]:
# hyperparameter
epochs = 1
log_interval = 100 # log 확인하는 interval

In [40]:
# 최종본

for epoch in range(1, epochs+1):
    # Train Mode로 변환
    model.train()
    
    # enumerate : idx도 같이 나오는 함수
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device) # cpu
        optimizer.zero_grad() # opt 초기화
        output = model(data) # data를 model에 넣음 > 예측치 :output
        loss = F.nll_loss(output, target) # loss 계산
        loss.backward() # gradient 계산
        optimizer.step() # w 업데이트
        
        # 중간에 잘 되고 있는지 log 확인
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset), # len 2개 = 현 data가 몇 번째 data인지, 전체 data의 크기
                100 * batch_idx / len(train_loader), loss.item() # 몇 % 돌아갔는지
            ))
            
        # 여기까지가 model 학습 !
        
    # 여기서부터 평가!                
    model.eval()

    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.367843
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.321535
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.550576
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.344955
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.282179
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.343969
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.209759
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.231541
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.372720
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.369015

Test set: Average Loss: 0.2817, Accuracy: 9208/10000 (92%)



# Evaluation

- 앞에서 model.train() 모드로 변한 것처럼 평가 할 때는 model.eval()로 설정
    - Batch Normalization이나 Drop Out 같은 Layer들을 잠금

In [23]:
model.eval() # evaluation mode

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

- autograd engine, 즉 backpropagatin이나 gradient 계산 등을 꺼서 memory usage를 줄이고 속도를 높임

In [27]:
test_loss = 0
correct = 0

# no_grad : back prop 안하게 함.
with torch.no_grad():
    data, target = next(iter(test_loader))
    data, target = data.to(device), target.to(device) # cpu
    output = model(data) # 모델에 넣어서 pred
    
    # log를 보기 위해서
    # sum : 1개의 스칼라로 변환됨. 누적을 보는 것
    # mean과 값 자체를 보는 none도 있음
    test_loss += F.nll_loss(output, target, reduction='sum').item() 
    
    # 제일 큰 max 값을 뽑음 > 제일 강한 위치의 idx > classification
    # dim=1 : 차원을 그대로 유지
    pred = output.argmax(dim=1, keepdim=True)
    
    # eq : 예측과 답이 같은지 아닌지 확인 > T/F로 나옴
    # sum : T끼리 F끼리 합침
    correct = pred.eq(target.view_as(pred)).sum().item()

In [28]:
output.shape

torch.Size([64, 10])

In [29]:
pred.shape # dim은 유지, 10개 중에 1개롤 class 선정됨

torch.Size([64, 1])

In [30]:
# pred와 같은 크기
target.shape

torch.Size([64])

In [37]:
target.view_as(pred)

tensor([[5],
        [9],
        [5],
        [6],
        [7],
        [8],
        [6],
        [3],
        [0],
        [2],
        [3],
        [7],
        [9],
        [5],
        [3],
        [3],
        [4],
        [2],
        [8],
        [0],
        [9],
        [7],
        [9],
        [4],
        [8],
        [9],
        [1],
        [5],
        [2],
        [5],
        [0],
        [7],
        [1],
        [5],
        [2],
        [5],
        [1],
        [8],
        [7],
        [7],
        [4],
        [1],
        [5],
        [9],
        [6],
        [4],
        [0],
        [6],
        [8],
        [9],
        [4],
        [5],
        [1],
        [4],
        [1],
        [4],
        [2],
        [2],
        [3],
        [3],
        [4],
        [1],
        [6],
        [1]])

In [38]:
target.view_as(pred).shape

torch.Size([64, 1])

In [31]:
# 64개 중 54개 맞춘 것
pred.eq(target.view_as(pred)).sum()

tensor(54)

In [32]:
pred.eq(target.view_as(pred)).sum().item() / 64

0.84375

In [33]:
# 틀린 것
test_loss

28.772645950317383

In [34]:
# 맞춘 것
correct

54

In [35]:
# 전체로 나눠서 비율을 구함
test_loss /= len(test_loader.dataset)

In [36]:
test_loss

0.0028772645950317383

In [39]:
model.eval()  # evaluation

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_loader: # for로 모든 test data를 넣음
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

# test 중에 나온 손실들의 평균 = 이 모델에서 나왔던 손실들의 평균
test_loss /= len(test_loader.dataset)

print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))


Test set: Average Loss: 0.4464, Accuracy: 8886/10000 (89%)

